In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from subprocess import call
from sklearn.metrics import mean_squared_error
import math

## Data Handling

In [2]:
# Load the data
df_full = pd.read_csv('/home/jj/CILProject22/data/raw/data_train.csv')
df_train = pd.read_csv('/home/jj/CILProject22/data/raw/cross_validation/train_split_4.csv')
df_test = pd.read_csv('/home/jj/CILProject22/data/raw/cross_validation/test_split_4.csv')

dic_full = {
    'user_id': [int(str(x).partition("_")[0][1:]) for x in df_full['Id']],
    'item_id': [int(str(x).partition("_")[2][1:]) for x in df_full['Id']],
    #'combined': [(str(x).partition("_")[0][1:],str(x).partition("_")[2][1:]) for x in df['Id']],
    'rating': [float(x) for x in df_full['Prediction']],
}
dic_train = {
    'user_id': [int(str(x).partition("_")[0][1:]) for x in df_train['Id']],
    'item_id': [int(str(x).partition("_")[2][1:]) for x in df_train['Id']],
    'rating': [float(x) for x in df_train['Prediction']],
}
dic_test = {
    'user_id': [int(str(x).partition("_")[0][1:]) for x in df_test['Id']],
    'item_id': [int(str(x).partition("_")[2][1:]) for x in df_test['Id']],
    'rating': [float(x) for x in df_test['Prediction']],
}

full_data = pd.DataFrame(dic_full)
train_data = pd.DataFrame(dic_train)
test_data = pd.DataFrame(dic_test)
full_data[:100]


,user_id,item_id,rating
0,44,1,4.0
1,61,1,3.0
2,67,1,4.0
3,72,1,3.0
4,86,1,5.0
...,...,...,...
95,2706,1,4.0
96,2820,1,3.0
97,2883,1,2.0
98,2939,1,3.0


In [3]:
n_users = 10000
n_items = 1000
ratings = np.zeros((n_users, n_items))
train = np.zeros((n_users, n_items))
test = np.zeros((n_users, n_items))

# Create train and test sets via Tobi
for row in full_data.itertuples(index = False):
    ratings[int(row.user_id) - 1, int(row.item_id) - 1] = int(row.rating)

for row in train_data.itertuples(index = False):
    train[int(row.user_id) - 1, int(row.item_id) - 1] = int(row.rating)

for row in test_data.itertuples(index = False):
    test[int(row.user_id) - 1, int(row.item_id) - 1] = int(row.rating)

# Create train and test sets randomly
def create_random_train_test(ratings):
    """
    split into training and test sets,
    remove 10 ratings from each user
    and assign them to the test set
    """
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_index = np.random.choice(
            np.flatnonzero(ratings[user]), size = 10, replace = True)

        train[user, test_index] = 0.0
        test[user, test_index] = ratings[user, test_index]
        
    # assert that training and testing set are truly disjoint
    assert np.all(train * test == 0)
    return train, test
    

# train, test = create_random_train_test(ratings)
# del ratings

print(ratings[:10, :10])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
 [0. 0. 0. 3. 0. 5. 0. 4. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 5. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0. 3.]
 [0. 0. 0. 1. 0. 5. 0. 5. 0. 0.]]


In [4]:
# normalization per item
train_T = np.transpose(train)
test_T = np.transpose(test)
print(train_T.shape)
avgs, stds = [], []
for item in range(n_items):
    mask = np.nonzero(train_T[item])
    mask_test = np.nonzero(test_T[item])
    avg = np.mean(train_T[item][mask])
    std = np.std(train_T[item][mask])
    avgs.append(avg)
    stds.append(std)
    train_T[item][mask] = train_T[item][mask] - avg / std
    test_T[item][mask_test] = test_T[item][mask_test] - avg / std


train = np.transpose(train_T)
test_norm = np.transpose(test_T)
print(train.shape)
print(train[:10][:10])

(1000, 10000)
(10000, 1000)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  2.34660803 -0.08553854
  -0.45348009]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


## Model

In [5]:
# Todo:
# - change n_iter to some stop condition (often converges after 10 iterations)

class ExplicitMF:
    """
    Train a matrix factorization model using Alternating Least Squares
    to predict empty entries in a matrix
    
    Parameters
    ----------
    n_iters : int
        number of iterations to train the algorithm
        
    n_factors : int
        number of latent factors to use in matrix 
        factorization model, some machine-learning libraries
        denote this as rank
        
    reg : float
        regularization term for item/user latent factors,
        since lambda is a keyword in python we use reg instead
    """

    def __init__(self, n_iters, n_factors, reg):
        self.reg = reg
        self.n_iters = n_iters
        self.n_factors = n_factors  
        self.n_user, self.n_item = train.shape
        self.user_factors = np.random.random((self.n_user, self.n_factors))
        self.item_factors = np.random.random((self.n_item, self.n_factors))
        
        
    def fit(self, train, test):
        """
        pass in training and testing at the same time to record
        model convergence, assuming both dataset is in the form
        of User x Item matrix with cells as ratings
        """

        self.test_rmse_record  = []
        self.train_rmse_record = []   
        for i in range(self.n_iters):
            print('iteration: ', i)
            self.user_factors = self._als_step(train, self.user_factors, self.item_factors)
            self.item_factors = self._als_step(train.T, self.item_factors, self.user_factors)
            predictions = self.predict()
            test_rmse = self.compute_rmse(test, predictions)
            train_rmse = self.compute_rmse(train, predictions)
            self.test_rmse_record.append(test_rmse)
            self.train_rmse_record.append(train_rmse)
        
        return self    
    
    def _als_step(self, ratings, solve_vecs, fixed_vecs):
        """
        when updating the user matrix,
        the item matrix is the fixed vector and vice versa
        """
        for i in range(solve_vecs.shape[0]):
            A = np.zeros((self.n_factors, self.n_factors))
            b = np.zeros((self.n_factors,))
            for j in range(fixed_vecs.shape[0]):
                if ratings[i, j] != 0:
                    A += np.outer(fixed_vecs[j], fixed_vecs[j])
                    b += ratings[i][j] * fixed_vecs[j]
            A += self.reg * np.eye(self.n_factors)
            solve_vecs[i] = np.linalg.solve(A, b).T

        # A = fixed_vecs.T.dot(fixed_vecs) + np.eye(self.n_factors) * self.reg
        # b = ratings.dot(fixed_vecs)
        # A_inv = np.linalg.inv(A)
        # solve_vecs = b.dot(A_inv)
        
        return solve_vecs
    
    def predict(self):
        """predict ratings for every user and item"""
        pred = self.user_factors.dot(self.item_factors.T)
        return pred
    
    @staticmethod
    def compute_rmse(y_true, y_pred):
        """ignore zero terms prior to comparing the mse"""
        mask = np.nonzero(y_true)
        mse = mean_squared_error(y_true[mask], y_pred[mask])
        return math.sqrt(mse)

In [6]:
model = ExplicitMF(n_iters = 20, n_factors = 30, reg = 5.0)
model.fit(train, test_norm)
print(model.train_rmse_record)

iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
[0.8902304926618374, 0.8001531644719705, 0.7750507924082778, 0.7610531248988119, 0.7523759052999015, 0.7465360105810963, 0.7423490569613881, 0.7391985513086182, 0.7367430556085567, 0.7347818420896453, 0.733188442381822, 0.731876608867411, 0.7307833568197495, 0.7298608160311583, 0.7290720681287165, 0.7283886277563836, 0.7277886238474904, 0.7272553577270584, 0.7267761400371815, 0.7263413625809525]


In [7]:
# print(model.test_mse_record)
pred = model.predict()
pred_T = np.transpose(pred)
for item in range(n_items):
    pred_T[item] = pred_T[item] * stds[item] + avgs[item]
    for i in range(len(pred_T[item])):
        if pred_T[item][i] < 1:
            pred_T[item][i] = 1
        elif pred_T[item][i] > 5:
            pred_T[item][i] = 5

pred = np.transpose(pred_T)

print(f'RMSE: {model.compute_rmse(test, pred)}')
print('\n', pred[:10][:10])
print('\n', train[:10][:10])

RMSE: 4.067915964825051

 [[3.16035093 3.45941034 3.18988747 ... 3.89378716 3.36017374 4.72132228]
 [2.98456671 3.28455244 2.57112525 ... 4.58565665 3.76852347 3.52321185]
 [3.51797834 3.5457358  3.04169291 ... 4.50596075 2.5256685  4.39462024]
 ...
 [3.70627594 3.76761784 3.891213   ... 4.14847822 3.53037013 3.41513651]
 [3.51798063 3.16424224 3.6983676  ... 3.18026881 3.01121373 3.65492157]
 [3.14226943 3.03483047 2.18658741 ... 4.05816595 3.57652853 4.43191391]]

 [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  2.34660803 -0.08553854
  -0.45348009]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


## Cross Validation

In [8]:
# Big Cross Validation
regs = [0.5, 0.1, 0.05]
n_factors = [25, 30, 35]

best = (0, 0)
best_rmse = 100
for reg in regs:
    for n_factor in n_factors:
        model = ExplicitMF(n_iters = 100, n_factors = n_factor, reg = reg)
        model.fit(train, test)
        pred = model.predict()
        pred_T = np.transpose(pred)
        for item in range(n_items):
            pred_T[item] = pred_T[item] * stds[item] + avgs[item]
        pred = np.transpose(pred_T)
        rmse = model.compute_rmse(test, pred)
        if rmse < best_rmse:
            best_rmse = rmse
            best = (reg, n_factor)
            print(f'New best Hyperparameters: {best} with RMSE: {best_rmse}')


iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jj/CILProject22/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_343261/354650986.py", line 10, in <cell line: 7>
    model.fit(train, test)
  File "/tmp/ipykernel_343261/113108952.py", line 45, in fit
    self.item_factors = self._als_step(train.T, self.item_factors, self.user_factors)
  File "/tmp/ipykernel_343261/113108952.py", line -1, in _als_step
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jj/CILProject22/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/jj/CILProject22/venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  Fil

# Submission

In [ ]:
# normalize training data
ratings_T = np.transpose(ratings)
avgs_full, stds_full = [], []
for item in range(n_items):
    mask = np.nonzero(ratings_T[item])
    avg = np.mean(ratings_T[item][mask])
    std = np.std(ratings_T[item][mask])
    avgs_full.append(avg)
    stds_full.append(std)
    ratings_T[item][mask] = ratings_T[item][mask] - avg / std

ratings = np.transpose(ratings_T)

# train the model
print('\n', 'Training the model...')
model = ExplicitMF(n_iters = 200, n_factors = 30, reg = 0.1)
model.fit(ratings, test)

# predict ratings
pred = model.predict()
pred_T = np.transpose(pred)
for item in range(n_items):
    pred_T[item] = pred_T[item] * stds_full[item] + avgs_full[item]
pred = np.transpose(pred_T)
print(pred)

# write to submission-file
sample_sub = pd.read_csv("/home/dustin/Documents/Study/Master2/CILProject22/data_raw/sampleSubmission.csv")
prediction = []
for cell_id in sample_sub.Id:
    row, col = cell_id.split("_")
    prediction.append(pred[int(row[1:])-1, int(col[1:])-1])
sample_sub.Prediction = prediction
sample_sub.to_csv("../data/als.csv", index=False)
sample_sub

# Library Code

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    validation_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS().setMaxIter(maxIter).setRank(rank).setRegParam(reg)
            # train ALS model
            print(f'Training model with rank {rank} and regularization {reg}')
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {} with RSME = {}'.format(best_rank, best_regularization, min_error))
    return best_model


def make_recommendations(self, fav_movie, n_recommendations):
    """
    make top n movie recommendations
    Parameters
    ----------
    fav_movie: str, name of user input movie
    n_recommendations: int, top n recommendations
    """
    # get data
    movie_user_mat_sparse, hashmap = self._prep_data()
    # get recommendations
    raw_recommends = self._inference(
        self.model, movie_user_mat_sparse, hashmap,
        fav_movie, n_recommendations)
    # print results
    reverse_hashmap = {v: k for k, v in hashmap.items()}
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance '
              'of {2}'.format(i+1, reverse_hashmap[idx], dist))

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# from pyspark.sql import SQLContext
# sc = SparkContext.getOrCreate()
# sqlContext = SQLContext(sc)

# spark_dff = sqlContext.createDataFrame(train_data)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/21 02:54:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
train_lib = list(zip(train_data.user_id, train_data.item_id, train_data.rating))
test_lib = list(zip(test_data.user_id, test_data.item_id, test_data.rating))
full_lib = train_lib + test_lib
print(train_lib[:10])
print(test_lib[:10])

df_train = spark.createDataFrame(train_lib, ["user", "item", "rating"])
df_test = spark.createDataFrame(test_lib, ["user", "item", "rating"])
df_full = spark.createDataFrame(full_lib, ["user", "item", "rating"])
df = spark.createDataFrame([(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)], ["user", "item", "rating"])
#print(df_train.head())

[(44, 1, 4.0), (67, 1, 4.0), (72, 1, 3.0), (86, 1, 5.0), (90, 1, 4.0), (108, 1, 3.0), (114, 1, 3.0), (135, 1, 5.0), (152, 1, 4.0), (165, 1, 3.0)]
[(61, 1, 3.0), (120, 1, 2.0), (457, 1, 2.0), (670, 1, 3.0), (966, 1, 5.0), (1107, 1, 4.0), (1338, 1, 3.0), (1583, 1, 4.0), (1622, 1, 5.0), (1789, 1, 3.0)]


In [6]:
# only for manual testing
als = ALS().setMaxIter(20).setRank(20).setRegParam(0.1)
# train ALS model
print(f'Training model with rank {20} and regularization {5}')
model = als.fit(df_train)
# evaluate the model by computing the RMSE on the validation data

Training model with rank 20 and regularization 5
22/07/21 02:55:28 WARN TaskSetManager: Stage 0 contains a task of very large size (15398 KiB). The maximum recommended task size is 1000 KiB.


22/07/21 02:55:33 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


22/07/21 02:55:33 WARN TaskSetManager: Stage 1 contains a task of very large size (15398 KiB). The maximum recommended task size is 1000 KiB.


22/07/21 02:55:57 ERROR Instrumentation: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1510)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1433)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1179)
	at java.base/java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1553)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1510)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1433)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1179)
	at java.base/java.io.ObjectOutputStream.writeArray(ObjectOutputStream.java:1379)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1175)
	at java.base/java.io.Objec

Py4JJavaError: An error occurred while calling o89.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1510)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1433)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1179)
	at java.base/java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1553)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1510)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1433)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1179)
	at java.base/java.io.ObjectOutputStream.writeArray(ObjectOutputStream.java:1379)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1175)
	at java.b

In [ ]:
predictions = model.transform(df_train)
print(predictions.show())

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print('RMSE on test data = {}'.format(rmse))

In [ ]:
model = tune_ALS(df_train, df_test, maxIter = 20, regParams = [0.05, 0.1, 0.5], ranks = [10, 20, 30])
print('Training complete')
pred = model.transform(df_test)
# pred_T = np.transpose(pred.select('prediction').collect())
# for item in range(n_items):
#     pred_T[item] = pred_T[item] * stds_full[item] + avgs_full[item]
# pred = np.transpose(pred_T)
# print(pred)


evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
rmse = evaluator.evaluate(pred)
print('RSME: {}' .format(rmse))

# Submission Library

In [ ]:
# train ALS model
als = ALS().setMaxIter(3).setRank(30).setRegParam(0.1)
print(f'Training model with rank {30} and regularization {0.1}')
model = als.fit(df_full)

# predict ratings
sample_sub = pd.read_csv("/home/dustin/Documents/Study/Master2/CILProject22/data_raw/sampleSubmission.csv")
to_predict = []
for cell_id in sample_sub.Id:
    row, col = cell_id.split("_")
    to_predict.append((int(row[1:]), int(col[1:]), 0.0))
df_to_pred = spark.createDataFrame(to_predict, ["user", "item", "rating"])
predictions = model.transform(df_to_pred)
print(predictions.show())
prediction = [pred[0] for pred in list(predictions.select('prediction').collect())]

# write to csv
sample_sub.Prediction = prediction
sample_sub.to_csv("../data/als.csv", index=False)
sample_sub